# <center> Macro Monitor </center>

- **Summary:** This notebook provides a macro monitor that can daily send e-mails with a PDF macro report. The report includes data only in an yearly frequency. 
$$
$$
- **Data Included in the Report:**
$$
$$ 
    - BLS US Inflation data.
    - World Equity Indices data.
    - FX USD data.
    - Commodities futures data.

### Importing necessary modules:

In [219]:
import requests
import json
from functools import reduce
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import math
import fpdf
import pandas as pd
from PyPDF2 import PdfFileMerger, PdfFileReader
import dataframe_image
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import pandas_datareader as pdr

In [220]:
from socket import gethostname
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib
import json

### US BLS Inflation Data:

In [130]:
series_names = {'CUSR0000SA0':'All items','CUSR0000SA0L1E':'All items less food and energy',
                'CUSR0000SAF1':'Food', 'CUSR0000SA0E':'Energy', 'CUSR0000SAA':'Apparel', 
                'CUSR0000SAE':'Education and communication','CUSR0000SAG':'Other goods and services',
               'CUSR0000SAM':'Medical care','CUSR0000SAR':'Recreation','CUSR0000SAT':'Transportation'}

In [131]:
id_list = ['CUSR0000SA0','CUSR0000SA0L1E','CUSR0000SAF1',
            'CUSR0000SA0E', 'CUSR0000SAA','CUSR0000SAE',
            'CUSR0000SAG','CUSR0000SAM','CUSR0000SAR',
            'CUSR0000SAT']

In [132]:
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": id_list,"startyear":"2014", "endyear":"2022"})

In [133]:
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)

**Constructing a list of dataframes with one dataframe for each series:**

In [134]:
data_list = [] # criando uma lista de dataframes
for series in json_data["Results"]["series"]:
    df = pd.DataFrame(series["data"]).drop(["footnotes","periodName","latest"],axis=1) # 
    df.rename(columns={"value":series_names[series["seriesID"]]}, inplace=True) # renomeando a coluna do dataframe
    data_list.append(df) # inserindo o dataframe na lista de dataframes

In [135]:
data = reduce(lambda left,right: pd.merge(left,right,on=["year","period"]),data_list) # juntando os dataframes da lista

**Modifying the data and the dataframe index:**

In [136]:
df_data = data.iloc[::-1].reset_index().drop("index",axis=1) # invertendo os dados.

In [137]:
df_data["month"] = df_data["period"].apply(lambda x: x[1:]) # adicionando o mês.
df_data["day"] = (df_data["month"]!="").astype(int) # adicionando um dia falso com 1o.

In [138]:
index = pd.to_datetime(df_data[["year","month","day"]]).dt.strftime('%Y-%m') # criando datetime index.
df_data.index = index # trocado o index do dataframe
data = df_data.drop(["month","day","year","period"],axis=1).astype(float) # criando dados finais.
data.index = pd.to_datetime(data.index)
data.tail(5)

,All items,All items less food and energy,Food,Energy,Apparel,Education and communication,Other goods and services,Medical care,Recreation,Transportation
2022-03-01,287.708,288.811,295.809,305.469,127.044,143.680,498.538,538.665,129.282,266.004
2022-04-01,288.663,290.455,298.379,297.233,126.087,143.439,500.673,541.057,129.830,264.995
2022-05-01,291.474,292.289,301.879,308.839,126.927,143.363,502.966,543.220,130.304,270.297
2022-06-01,295.328,294.354,304.867,332.087,127.929,143.601,505.312,546.861,130.687,280.691
2022-07-01,295.271,295.275,308.220,316.955,127.818,143.356,507.204,549.282,131.107,274.821


In [139]:
index_series = data.copy()

**Inflation YoY:**

In [140]:
df = (index_series.resample("Y").last().pct_change()*100).dropna().round(decimals=2)
df.index = df.index.year
df_t = df.T
df_t =  df_t.sort_values(by=2022)

bls_table = df_t.style.background_gradient().set_table_styles([{'selector': 'th', 'props': [('font-size', '18px')]},
                                     {'selector': 'td', 'props': [('font-size', '16px')]}]).format(precision=2)
bls_table

,2015,2016,2017,2018,2019,2020,2021,2022
Education and communication,1.39,-0.23,-1.74,0.23,1.37,1.97,1.63,-0.27
Apparel,-0.47,0.11,-1.55,-0.08,-1.30,-4.17,5.71,2.98
Recreation,0.64,0.68,1.47,1.09,1.49,0.94,3.27,3.30
Medical care,2.55,3.93,1.77,2.02,4.57,1.78,2.17,3.39
All items less food and energy,2.07,2.20,1.77,2.20,2.23,1.60,5.48,3.69
Other goods and services,1.87,2.17,2.03,2.12,2.33,2.40,4.46,4.12
All items,0.64,2.05,2.13,1.92,2.26,1.28,7.10,5.41
Food,0.80,-0.19,1.61,1.57,1.80,3.92,6.27,7.23
Transportation,-4.60,2.27,3.61,0.72,1.92,-2.64,21.28,9.88
Energy,-12.99,4.93,7.05,-0.44,3.36,-7.34,29.42,20.26


**Saving:**

In [141]:
dataframe_image.export(bls_table,"Images/BLS Inflation Table.png")

### World Equity Indices:

In [143]:
eq_indices  =   {"US (S&P500)":"^GSPC","US (Dow Jones)":"^DJI",  "US (Nasdaq)":"^IXIC", "US (NYSE composite)":"^NYA",
                 "UK (CBOE BUK 100)":"^BUK100P", "UK (FTSE)":"^FTSE","Germany (DAX)":"^GDAXI","France (CAC)":"^FCHI", 
                 "Europe (Euronext)":"^N100","Singapore (STI)":"^STI","Australia (ASX)":"^AXJO", 
                 "Indonesia (Jakarta Composite)":"^JKSE","Malaysia (Bursa)":"^KLSE","South Korea (Kospi)":"^KS11",
                 "Brazil (IBOV)":"^BVSP","Argentina (Merval)":"^MERV","China (Shanghai)":"000001.SS",
                 "China (Shenzhen)":"399001.SZ", "Belgium (Bel 200)":"^BFX","Japan (Nikkei)":"^N225",
                 "Russia (MOEX)":"IMOEX.ME","Hong Kong (Hang Seng)":"^HSI"}

In [144]:
equity_data = pd.DataFrame()
for key, value in eq_indices.items():
    try:
        equity_data[key] = pdr.DataReader(value, data_source = "yahoo", start=2010)["Close"]
    except:
        print("Error in: ", key)

Error in:  UK (FTSE)
Error in:  Singapore (STI)
Error in:  Malaysia (Bursa)


In [145]:
equity_tab = (equity_data.resample("Y").last().pct_change().dropna(how="all")*100).round(2)
equity_tab.index = equity_tab.index.year
equity_table = equity_tab.T.copy()
equity_table =  equity_table.sort_values(by=2022)
eq_table = equity_table.style.background_gradient().set_table_styles([{'selector': 'th', 'props': [('font-size', '18px')]},
                                                 {'selector': 'td', 'props': [('font-size', '16px')]}]).format(precision=2)
eq_table

Date,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Russia (MOEX),nan,nan,nan,-3.61,21.55,26.76,-5.51,11.79,29.14,7.98,15.15,-43.30
US (Nasdaq),-1.80,15.91,38.32,13.40,5.73,7.50,28.24,-3.88,35.23,43.64,21.39,-16.60
China (Shenzhen),-28.41,2.22,-10.91,35.62,14.98,-19.64,8.48,-34.42,44.08,38.73,2.67,-16.41
South Korea (Kospi),-10.98,9.38,0.72,-4.76,2.39,3.32,21.76,-17.28,7.67,30.75,3.63,-15.10
Hong Kong (Hang Seng),-19.97,22.91,2.87,1.28,-7.16,0.39,35.99,-13.61,9.07,-3.40,-14.08,-13.77
Germany (DAX),-14.69,29.06,25.48,2.65,9.56,6.87,12.51,-18.26,25.48,3.55,15.79,-13.15
Belgium (Bel 200),-19.58,18.38,18.55,12.36,13.66,-3.42,10.30,-18.46,21.96,-8.46,19.02,-10.81
US (S&P500),-0.00,13.41,29.60,11.39,-0.73,9.54,19.42,-6.24,28.88,16.26,26.89,-10.20
China (Shanghai),-21.68,3.17,-6.75,52.87,9.41,-12.31,6.56,-24.59,22.30,13.87,4.80,-9.97
France (CAC),-17.94,14.57,18.66,-0.54,9.46,3.96,9.26,-10.95,26.37,-7.14,28.85,-8.38


**Saving:**

In [146]:
dataframe_image.export(eq_table,"Images/Equity Indices Table.png")

### FX Rates:

In [147]:
fx_tickers = {"Brazil (USD/BRL)":"USDBRL=X","Canada (USD/CAD)":"USDCAD=X","Argentina (USD/ARG)":"USDARG=X",
              "India (USD/INR)":"USDINR=X","Colombia (USD/COP)":"USDCOP=X","Sweden (USD/SEK)":"USDSEK=X",
              "Europe (USD/EUR)":"USDEUR=X","England (USD/GBP)":"USDGBP=X"}

In [148]:
fx_data = pd.DataFrame()
for key, value in fx_tickers.items():
    try:
        fx_data[key] = pdr.DataReader(value, data_source = "yahoo", start=2010)["Close"]
    except:
        print("Error in: ", key)

Error in:  Argentina (USD/ARG)


In [149]:
fx_tab = (fx_data.resample("Y").last().pct_change().dropna(how="all")*100).round(2)
fx_tab.index = fx_tab.index.year
fx_table = fx_tab.T.copy()
fx_table =  fx_table.sort_values(by=2022)
currency_usd_table = fx_table.style.background_gradient().set_table_styles([{'selector': 'th', 
                  'props': [('font-size', '18px')]}, {'selector': 'td', 'props': [('font-size', '16px')]}]).format(precision=2)
currency_usd_table

Date,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Brazil (USD/BRL),12.23,9.65,15.36,12.50,49.08,-17.80,1.81,17.01,3.65,29.26,7.31,-8.96
Canada (USD/CAD),2.07,-2.54,7.05,9.02,19.54,-2.84,-6.77,8.41,-4.12,-2.37,-0.05,0.22
Colombia (USD/COP),1.10,-8.78,9.23,23.12,32.13,-5.52,0.58,8.82,1.17,4.18,18.95,2.16
India (USD/INR),18.28,3.31,12.83,2.59,4.86,2.22,-5.70,9.15,1.99,2.56,1.77,7.01
Europe (USD/EUR),2.66,-2.05,-4.22,13.55,11.23,3.39,-11.44,4.39,2.12,-8.92,8.61,9.77
England (USD/GBP),0.14,-4.62,-2.01,5.95,5.06,20.60,-8.58,5.89,-3.20,-3.80,0.98,10.68
Sweden (USD/SEK),2.12,-5.65,-1.39,20.92,8.48,7.81,-9.05,8.91,4.06,-12.37,10.48,11.20


**Saving:**

In [150]:
dataframe_image.export(currency_usd_table,"Images/FX Table.png")

### Commodities (Futures):

In [151]:
commodities_tickers ={"Gold":"GC=F", "Silver":"SI=F","Crude Oil":"CL=F","Corn":"ZC=F","Oat":"ZO=F","Soybeans":"ZS=F",
                      "Live Cattle":"LE=F", "Cocoa":"CC=F", "Coffee":"KC=F", "Cotton":"CT=F", "Orange Juice":"OJ=F",
                      "Sugar":"SB=F","Natural Gas":"NG=F","Platinum":"PL=F","Copper":"HG=F","Palladium":"PA=F",
                     "Heating Oil":"HO=F", "Lean Hogs":"HE=F","Lumber":"LBS=F","Rough Rice":"ZR=F","Wheat":"KE=F"}

In [152]:
commodities_data = pd.DataFrame()
for key, value in commodities_tickers.items():
    try:
        commodities_data[key] = pdr.DataReader(value, data_source = "yahoo", start="2010",end="2022-07-22")["Close"]
    except:
        print("Error in: ", key)

In [153]:
commodities_tab = (commodities_data.resample("Y").last().pct_change().dropna(how="all")*100).round(2)
commodities_tab.index = commodities_tab.index.year
commodities_table = commodities_tab.T.copy()
commodities_table =  commodities_table.sort_values(by=2022)
comm_table = commodities_table.style.background_gradient().set_table_styles([{'selector': 'th', 'props': [('font-size', '18px')]},
                                                 {'selector': 'td', 'props': [('font-size', '16px')]}]).format(precision=2)
comm_table

Date,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Lumber,-18.18,51.32,-3.69,-8.05,-22.20,22.90,41.50,-25.78,21.89,115.42,31.47,-49.01
Oat,-21.45,12.28,1.94,-14.26,-28.48,5.18,5.47,14.21,6.09,23.54,89.33,-33.75
Copper,-22.71,6.11,-5.48,-17.52,-25.12,17.50,31.31,-19.87,6.32,25.77,26.78,-24.84
Silver,-9.82,8.24,-35.91,-19.51,-11.50,15.69,7.05,-9.54,15.52,47.70,-11.41,-20.33
Cotton,-36.61,-18.15,12.64,-28.79,4.99,11.65,11.30,-8.18,-4.36,13.14,44.14,-11.70
Platinum,-21.07,9.93,-10.89,-11.83,-26.24,1.11,3.62,-14.80,22.05,10.71,-10.32,-10.70
Cocoa,-30.51,6.02,21.15,7.42,10.34,-33.79,-11.01,27.70,5.13,2.48,-3.19,-8.85
Coffee,-5.68,-36.61,-23.02,50.50,-23.95,8.17,-7.92,-19.29,27.34,-1.12,76.30,-8.58
Gold,10.18,6.96,-28.24,-1.50,-10.44,8.46,13.59,-2.14,18.87,24.59,-3.47,-5.49
Sugar,-27.46,-16.27,-15.89,-11.52,4.96,28.02,-22.30,-20.65,11.55,15.42,21.89,-5.24


**Saving:**

In [154]:
dataframe_image.export(comm_table,"Images/Commodities Table.png")

**Adding the Tables to a PDF document:**

- **BLS Inflation:**

In [214]:
pdf = fpdf.FPDF()
pdf.add_page()
pdf.set_font("Times", "B", 14)
today = str(datetime.datetime.today().strftime('%Y-%m-%d'))
pdf.cell(1, 3, today, align='L')
pdf.set_font("Times", "B", 16)
pdf.cell(0, 30, 'BLS US Inflation Data',align = "C")
pdf.image('Images/BLS Inflation Table.png', x = 30, y = 40, h=75, w=150)
pdf.output("Report Drafts/BLS Inflation.pdf","F")

''

- **World Equity Indices:**

In [215]:
pdf = fpdf.FPDF()
pdf.add_page()
pdf.set_font("Times", "B", 14)
today = str(datetime.datetime.today().strftime('%Y-%m-%d'))
pdf.cell(1, 3, today, align='L')
pdf.set_font("Times", "B", 16)
pdf.cell(0, 30, 'World Equity Indices Data',align = "C")
pdf.image('Images/Equity Indices Table.png', x = 30, y = 40, w = 155, h=155)
pdf.output("Report Drafts/World Equity Indices.pdf","F")
pdf.add_page()

- **FX Table:**

In [216]:
pdf = fpdf.FPDF()
pdf.add_page()
pdf.set_font("Times", "B", 14)
today = str(datetime.datetime.today().strftime('%Y-%m-%d'))
pdf.cell(1, 3, today, align='L')
pdf.set_font("Times", "B", 16)
pdf.cell(0, 30, 'FX USD Data',align = "C")
pdf.image('Images/FX Table.png', x = 30, y = 40, w = 150, h=60)
pdf.output("Report Drafts/FX.pdf","F")
pdf.add_page()

- **Commodities:**

In [217]:
pdf = fpdf.FPDF()
pdf.add_page()
pdf.set_font("Times", "B", 14)
today = str(datetime.datetime.today().strftime('%Y-%m-%d'))
pdf.cell(1, 3, today, align='L')
pdf.set_font("Times", "B", 16)
pdf.cell(0, 30, 'Commodities Data',align = "C")
pdf.image('Images/Commodities Table.png', x = 30, y = 40, w = 150, h=150)
pdf.output("Report Drafts/Commodities.pdf","F")
pdf.add_page()

**Merging all PDF Files:**

In [218]:
merger = PdfFileMerger()
merger.append("Report Drafts/BLS Inflation.pdf")
merger.append("Report Drafts/World Equity Indices.pdf")
merger.append("Report Drafts/FX.pdf", 'rb')
merger.append("Report Drafts/Commodities.pdf", 'rb')
merger.write("Daily Macro Report - " + today + ".pdf")

**Sending E-mail with PDF Attachment:**

In [225]:
def SendMacroEmail(pdf_path, subject, destination_email, password):
    server = smtplib.SMTP('smtp.outlook.com', 587)
    server.starttls()
    server.login('market.automations@outlook.com', password)
    msg = MIMEMultipart()

    msg['Subject'] = subject
    msg['From'] = 'market.automations@outlook.com'
    msg['To'] = destination_email
    
    today = str(datetime.datetime.today().strftime('%Y-%m-%d'))
  
    with open(pdf_path, "rb") as f:
        attach = MIMEApplication(f.read(),_subtype="pdf")
        attach.add_header('Content-Disposition','attachment', filename=f"Macro Report - {today}")
        msg.attach(attach)
    server.send_message(msg)

In [228]:
SendMacroEmail("Daily Macro Report - " + today + ".pdf", f"Macro Report - {today}", "pedro.osoriomn@gmail.com", email_password)

**-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**